# 基本资料

In [36]:
L_total_parent = 225 # 母船总长度
Lpp_parent = 218 # 母船垂线间长
B_parent = 32.26 # 母船型宽
D_parent = 19.6 # 母船型深
d_design_parent = 12.5 # 母船设计吃水
d_structure_parent = 14.3 # 母船结构吃水\
DW_parent = 75088 # 母船载重量 :t
Lw_parent = Lpp_parent+5*0.75 # 水线间长:m ,首垂线与尾垂线距离+ 尾垂线后5个肋位，每个肋位距离为0.75m，参考母船总布置图
L_ball_parent = 4*0.75 # 首垂线至球首前端距，#256 - # 260，肋距为0.75m，参考母船总布置图
nabla_parent = 85488.56 # 母船结构吃水14.3m处的排水体积，参考静水力计算书
delta_parent = 87809.16 # 母船结构吃水处的排水量,参考静水力计算书吃水14.3m时的displt

In [37]:
Cb = 0.85
Lpp = 215.17    # m
B = 31.84       # m 
D = 19.34       # m
d = 14.11       # m
delta = 84292   # t

# 最小干舷计算

## 夏季最小干舷

### $f_0$

In [38]:
# 基本干舷
f0A = 2749+(2758-2749)*(Lpp-215) # 参考规范p182，插值获得
f0B = 3505 + (3520-3505)*(Lpp-215) # 参考规范p185，插值获得
f0 = f0B - (f0B-f0A)*0.6 # B-60船
f0 = int(f0)
print(f'''
基本干舷f0为{f0} mm
''')


基本干舷f0为3053 mm



### $f_1$
非$L<100m$的B型船,$f_1=0$

In [39]:
# 船长大于100m，f1为0
f1 = 0

### $f_2$
$$f_2=(f_0+f_1)(\frac{C_B+0.68}{1.36}-1)$$
当$C_B<=0.68$时，取$f_2=0$

In [40]:
# 方形系数修正
f2 = (f0+f1)*((Cb+0.68)/1.36-1)
f2 = int(f2)
print(f'方形系数修正值f2为{f2} mm')

方形系数修正值f2为381 mm


### $f_3$
$$f_3=(D_1-L/15)R$$
$$R = \left\{
\begin{aligned}
L/0.48,L<120m \\
250,L>=120m \\
\end{aligned}
\right.$$

In [41]:
# 型深修正
f3 = (D-Lpp/15)*250
f3 = int(f3)
print(f'型深修正值f3为{f3} mm')

型深修正值f3为1248 mm


### $f_4$
本船有一个首楼和一个桥楼，而上层建筑的长度S是位于船长L范围内的上层建筑的长度  
$$f_4=Kf_4'$$

首楼图像如下 
![首楼](img/首楼.png)
桥楼图像如下  
![桥楼](img/桥楼.png)

In [42]:
# 首楼
S1 = 12270 * Lpp/Lpp_parent # 按照长度比例缩放 
B1 = 1 # 由于首楼在宽度方向无收缩，因而宽度缩放因子为1
# 桥楼
S2 = 21500 *Lpp/Lpp_parent # 按照长度比例缩放
B2 = 22984 / 30645 # 上层建筑长度中点出的宽度比
# 上层建筑高于标准值，因此没有造成长度缩放
E1 = S1*B1
E2 = S2*B2
E = (E1+E2)/1000 # 有效上层建筑总长度，mm -> m
# 有效上层建筑占船长的比例
E_L = E/Lpp
print("有效上层建筑占船长的比例为: %.3f"%E_L)

有效上层建筑占船长的比例为: 0.130


In [43]:
# B-60 有首楼并有分立桥楼，参考课本p19表2.2.5
K = 6.3 + (12.7 - 6.3)*(E_L - 0.1)/0.1
print("系数K为:%.3f"%K,"%")

系数K为:8.236 %


In [44]:
# 船长L>=122m时，f4'=-1070mm
f4p = -1070
f4 = K/100 * f4p
f4 = int(f4)
print(f'f4为{f4} mm')

f4为-88 mm


### $f_5$

$$f_5=-(\frac{S_F+S_A}{2}+S)(0.75-\frac{l}{2L})$$
$l$是位于船长L范围内的封闭上层建筑的总长度(m)  
$S_F$是前半部舷弧的不足或多余数,$S_A$是后半段舷弧的不足或多余数  
$S=S_p+S_f$,$S_p$或$S_f=\frac{yL'}{3L}$，其中,$y$是尾垂线或首垂线处上层建筑的实际高度与标准高度之差(mm),$L'$是首楼或尾楼的平均封闭长度，取不大于$0.5L(m)$

首垂线舷弧数值如下  
![首垂线舷弧](img/首垂线舷弧.png)
尾垂线舷弧数值如下  
![尾垂线舷弧](img/尾垂线舷弧.png)
距离船尾L/3范围内的数值如下
![距离船尾](img/距离船尾.png)
距离船首L/3范围内的数值如下
![距离船首](img/距离船首.png)

In [45]:
# 舷弧修正
# 尾垂线
Lbp_real = 415 * D/D_parent # 尾垂线处的舷弧
Lbp16_real = 0 # 距尾垂线L/6处的舷弧
Lbp13_real = 0 # 距尾垂线L/3处的舷弧
Lmid_real = 0 # 距尾垂线L/2处的舷弧
# 首垂线
Lfp_real = 2200 * D/D_parent # 首垂线处的舷弧
Lfp16_real = 617 * D/D_parent # 距首垂线L/6处的舷弧
Lfp13_real = 0 # 距首垂线L/3处的舷弧
print(f"""
尾垂线舷弧大小为               {int(Lbp_real+0.5)}mm
距离尾垂线L/6处的舷弧大小为    {Lbp16_real}mm
距离尾垂线L/3处的舷弧大小为    {Lbp13_real}mm
距离尾垂线L/2处的舷弧大小为    {Lmid_real}mm
首垂线舷弧大小为               {int(Lfp_real+0.5)}mm
距离首垂线L/6处的舷弧大小为    {int(Lfp16_real+0.5)}mm
距离首垂线L/3处的舷弧大小为    {Lfp13_real}mm
""")


尾垂线舷弧大小为               409mm
距离尾垂线L/6处的舷弧大小为    0mm
距离尾垂线L/3处的舷弧大小为    0mm
距离尾垂线L/2处的舷弧大小为    0mm
首垂线舷弧大小为               2171mm
距离首垂线L/6处的舷弧大小为    609mm
距离首垂线L/3处的舷弧大小为    0mm



In [46]:
# 参考课本p20
Lbp_std = 25*(Lpp/3 + 10)
Lbp16_std  = 11.1*(Lpp/3+10)
Lbp13_std = 2.8*(Lpp/3+10)
Lmid_std = 0
Lfp_std = 2*Lbp_std    
Lfp16_std = 2*Lbp16_std
Lfp13_std = 2*Lbp13_std

In [47]:
SA = 1*(Lbp_real - Lbp_std) + 3*(Lbp16_real-Lbp16_std) + 3*(Lbp13_real-Lbp13_std) + 1*(Lmid_real-Lmid_std)
SA /= 8
SA = round(SA,2)
SF = 1*(Lfp_real - Lfp_std) + 3*(Lfp16_real-Lfp16_std) + 3*(Lfp13_real-Lfp13_std) + 1*(Lmid_real-Lmid_std)
SF /= 8
SF = round(SF,2)
print(f"""
SA大小为:   {SA}mm
SF大小为:   {SF}mm
""")


SA大小为:   -630.18mm
SF大小为:   -863.08mm



首垂线上层建筑高度如下
![首垂线上层建筑](img/首垂线上层建筑.png)

In [48]:
# 首垂线处上层建筑高度
Hfp = 3318 * D/D_parent # 根据型深比缩放
Hfp_std = 2300 # 课本p16,L>=125m,其他上层建筑类
y = Hfp - Hfp_std
L_ = E1/1000 # E1为首楼上层建筑有效长度;mm -> m
Sf = y*L_/(3*Lpp)
Sf = round(Sf,2)
# 尾垂线无上层建筑，因而y为0，Sp为0
Sp = 0
S = round(Sp + Sf,2)
print(f"""
Sp大小为   {Sp}mm
Sf大小为   {Sf}mm
S大小为    {S}mm
""")


Sp大小为   0mm
Sf大小为   18.27mm
S大小为    18.27mm



In [49]:
# 船长范围内上层建筑总有效长度
l = (E1+E2)/1000 # mm->m
print("船长范围内上层建筑总有效长度为%.2fm"%l)

船长范围内上层建筑总有效长度为28.03m


In [50]:
f5 = -((SF+SA)/2 + S)*(0.75 - l/2/Lpp)
f5 = round(f5,2)
print("f5值为:%.2f mm"%f5)

f5值为:498.83 mm


### 汇总

In [51]:
print(f"""
f0值为:    {f0}mm
f1值为:    {f1}mm
f2值为:    {f2}mm
f3值为:    {f3}mm
f4值为:    {f4}mm
f5值为:    {f5}mm
""")


f0值为:    3053mm
f1值为:    0mm
f2值为:    381mm
f3值为:    1248mm
f4值为:    -88mm
f5值为:    498.83mm



In [52]:
# 夏季最小干舷
Fmin = f0 + f1 + f2 + f3 + f4 + f5
print("夏季最小干舷为:%.2f mm"%Fmin)

夏季最小干舷为:5092.83 mm


In [53]:
# 实际干舷
Freal = round((D-d)*1000,2)
print(f"""
实际干舷大小为:          {Freal}mm
满足夏季最小干舷要求:    {Freal>=Fmin}
""")


实际干舷大小为:          5230.0mm
满足夏季最小干舷要求:    True



## 最小船首高度
$h_b=(6075L_{100}-1875L_{100}^2+200L_{100}^3)\times[2.08+0.609C_B-1.603C_{WF}-0.0129(L/d_1)]$  
$C_{WF}$是0.85$D$水线的前半体($L/2$)水线面面积系数,$d_1=0.85D$

16m水线面图如下
![16m水线](img/16水线.png)
17m水线面图如下
![17m水线](img/17水线.png)

In [54]:
from scipy.integrate import simpson
# 前体水线面面积系数，参考母船
# 辛普森法，以下单位均为mm
# 16m水线处
delta = 5450
tmpB = 16130 # 16m水线处的半宽值:mm
H_list = [15962,15458,14469,12749,10158,6337,152] # #17-#20,间隔0.5站
A1 = simpson(H_list,dx=delta)
A2 = delta*2*7*tmpB # #10-#17，近似为矩形
A = (A1 + A2)*2/1e6
tmpB /= 1000
Cwf1 = A/(Lpp_parent/2*tmpB*2)
Cwf1 = round(Cwf1,3)
# 17m水线处
tmpB = 16130 # mm
H_list = [16030,15613,14716,13114,10623,6762,562]# #17-#20,间隔0.5站
A1 = simpson(H_list,dx=delta)
A2 = delta*2*7*tmpB # #10-#17，近似为矩形
A = (A1 + A2)*2/1e6
tmpB /= 1000
Cwf2 = A/(Lpp_parent/2*tmpB*2)
Cwf2 = round(Cwf2,3)
# 母船0.85D即16.66m水线处的前半水线面系数为
Cwf = Cwf1 + (Cwf2-Cwf1)*(16.66-16)/1
Cwf = round(Cwf,3)
print("0.85D处的前半水线面系数为:%.3f"%Cwf)

0.85D处的前半水线面系数为:0.914


In [55]:
L100 = Lpp/100
d1 = 0.85*D
Fb = (6075 * L100 - 1875 * L100**2 + 200 * L100**3) * (2.08 + 0.609*Cb - 1.603*Cwf - 0.0129*(Lpp/d1))
Fb = int(Fb+0.5)
print(f'最小船首高度为{Fb} mm')

最小船首高度为6151 mm


法规规定，如果要求的船首高度是用舷弧达到的，则舷弧自首垂线起至少延伸到船长的15%处，如果是设置上层建筑达到的，则上层建筑至少延伸到首垂线后0.07L处

![首楼2](img/首楼.png)

In [56]:
# 如果以上层建筑来达到
Lsl = 12270 # 首楼长度:mm
print("满足要求: ",Lsl/1000>=0.07*Lpp)

满足要求:  False


![舷弧](img/舷弧.png)

In [57]:
# 如果以舷弧来达到
Lxh = 60139 # mm
print("满足要求:",Lxh/1000>=0.15*Lpp)

满足要求: True


In [58]:
# 自首垂线夏季最小干舷水线至露天甲板上边垂直距离
H1 = 2200 * D / D_parent # 根据母船首垂线舷弧缩放
H = H1 + Fmin # 船首高度
print("满足船首最小高度要求:",H>Fb)

满足船首最小高度要求: True


## 储备浮力

测投影面积应不小于  
$$
F_A = \frac{L}{1000}\{0.15[f_0(\frac{C_B+0.68}{1.36}-1)+f_3]+4(\frac{L}{3}+10)\}
$$

In [59]:
FA = Lpp/1000 * (0.15 * (f0*((Cb+0.68)/1.36-1)+f3) + 4*(Lpp/3+10))
FA = round(FA,2)
print("测投影面积最小为:%.2f m2"%FA)

测投影面积最小为:122.93 m2


![A2](img/A2.png)

![A3](img/A3.png)

In [60]:
# 参考课本p21
A1 = 0.15*Lpp * Fmin/1000 # m2
# 将A2分割为15份，每份间隔0.01L=2180mm
H2_list = [712,769,828,948,1051,1150,1230,1318,1453,1564,1671,1764,1866,1975,2081,2200]
dx = 2180 # mm
A2 = simpson(H2_list,dx=dx)/1e6
A2 = A2 * D * Lpp / D_parent / Lpp_parent # 母船数据缩放到子船
# 将A3分割为5份，每份间隔2454mm
H3_list = [2015,2030,2100,2143,2176,2244]
dx = 2454 # mm
A3 = simpson(H3_list,dx=dx)/1e6
A3 = A3 * D * Lpp / D_parent / Lpp_parent

A = round(A1+A2+A3,2)
print(f"""
测投影面积为            {A} m2
满足首部储备浮力要求    {A>FA}
""")


测投影面积为            234.48 m2
满足首部储备浮力要求    True

